In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import pymongo
import sys
import json
import os

In [3]:
df=pd.read_csv('Motor_vehicle_insurance_data.csv',sep=';')
for col in df.columns:
    if df[col].isnull().sum()>0:
        print(f"Column {col} has {df[col].isnull().sum()} null values")

Column Date_lapse has 70408 null values
Column Type_fuel has 1764 null values
Column Length has 10329 null values


C:\Users\joshi\AppData\Local\Temp\ipykernel_15956\2482879030.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('Motor_vehicle_insurance_data.csv',sep=';')


In [4]:
today = pd.Timestamp.today()
df['Date_lapse']=df['Date_lapse'].fillna(today)

In [53]:
df['Length']=df['Length'].fillna(df['Length'].mean())
df['Type_fuel']=df['Type_fuel'].fillna(df['Type_fuel'].mode()[0])


In [54]:
df.shape

(105555, 30)

In [55]:
df.head()

,ID,Date_start_contract,Date_last_renewal,Date_next_renewal,Date_birth,Date_driving_licence,Distribution_channel,Seniority,Policies_in_force,Max_policies,...,Area,Second_driver,Year_matriculation,Power,Cylinder_capacity,Value_vehicle,N_doors,Type_fuel,Length,Weight
0,1,05/11/2015,05/11/2015,05/11/2016,15/04/1956,20/03/1976,0,4,1,2,...,0,0,2004,80,599,7068.0,0,P,4.252007,190
1,1,05/11/2015,05/11/2016,05/11/2017,15/04/1956,20/03/1976,0,4,1,2,...,0,0,2004,80,599,7068.0,0,P,4.252007,190
2,1,05/11/2015,05/11/2017,05/11/2018,15/04/1956,20/03/1976,0,4,2,2,...,0,0,2004,80,599,7068.0,0,P,4.252007,190
3,1,05/11/2015,05/11/2018,05/11/2019,15/04/1956,20/03/1976,0,4,2,2,...,0,0,2004,80,599,7068.0,0,P,4.252007,190
4,2,26/09/2017,26/09/2017,26/09/2018,15/04/1956,20/03/1976,0,4,2,2,...,0,0,2004,80,599,7068.0,0,P,4.252007,190


In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105555 entries, 0 to 105554
Data columns (total 30 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   ID                    105555 non-null  int64  
 1   Date_start_contract   105555 non-null  object 
 2   Date_last_renewal     105555 non-null  object 
 3   Date_next_renewal     105555 non-null  object 
 4   Date_birth            105555 non-null  object 
 5   Date_driving_licence  105555 non-null  object 
 6   Distribution_channel  105555 non-null  object 
 7   Seniority             105555 non-null  int64  
 8   Policies_in_force     105555 non-null  int64  
 9   Max_policies          105555 non-null  int64  
 10  Max_products          105555 non-null  int64  
 11  Lapse                 105555 non-null  int64  
 12  Date_lapse            105555 non-null  object 
 13  Payment               105555 non-null  int64  
 14  Premium               105555 non-null  float64
 15  

In [16]:
data=df.to_dict(orient='records')
DB_NAME = "VehicleInsurance"
COLLECTION_NAME = "Proj1-Data"
CONNECTION_URL = "mongodb+srv://joshisparsh:url"

# above, either remove your credentials or delete the mongoDB resource bofore pushing it to github.

In [34]:
client = pymongo.MongoClient(CONNECTION_URL)
data_base = client[DB_NAME]
collection = data_base[COLLECTION_NAME]

In [47]:
from pymongo.errors import AutoReconnect
import time

MAX_RETRIES = 5
RETRY_DELAY = 3
batch_size = 50

for i in range(0, len(data), batch_size):
    batch = data[i:i+batch_size]
    for attempt in range(3):  # Retry max 3 times
        try:
            collection.insert_many(batch)
            print(f"✅ Inserted batch {i//batch_size + 1}")
            time.sleep(0.3)
            break
        except AutoReconnect as e:
            print(f"⚠️ Retry {attempt + 1} for batch {i//batch_size + 1}: {e}")
            time.sleep(2)
        except Exception as e:
            print(f"❌ Failed batch {i//batch_size + 1}: {e}")
            break


✅ Inserted batch 1
✅ Inserted batch 2
✅ Inserted batch 3
✅ Inserted batch 4
✅ Inserted batch 5
✅ Inserted batch 6
✅ Inserted batch 7
✅ Inserted batch 8
✅ Inserted batch 9
✅ Inserted batch 10
✅ Inserted batch 11
✅ Inserted batch 12
✅ Inserted batch 13
✅ Inserted batch 14
✅ Inserted batch 15
✅ Inserted batch 16
✅ Inserted batch 17
✅ Inserted batch 18
✅ Inserted batch 19
✅ Inserted batch 20
✅ Inserted batch 21
✅ Inserted batch 22
✅ Inserted batch 23
✅ Inserted batch 24
✅ Inserted batch 25
✅ Inserted batch 26
✅ Inserted batch 27
✅ Inserted batch 28
✅ Inserted batch 29
✅ Inserted batch 30
✅ Inserted batch 31
✅ Inserted batch 32
✅ Inserted batch 33
✅ Inserted batch 34
✅ Inserted batch 35
✅ Inserted batch 36
✅ Inserted batch 37
✅ Inserted batch 38
✅ Inserted batch 39
✅ Inserted batch 40
✅ Inserted batch 41
✅ Inserted batch 42
✅ Inserted batch 43
✅ Inserted batch 44
✅ Inserted batch 45
✅ Inserted batch 46
✅ Inserted batch 47
✅ Inserted batch 48
✅ Inserted batch 49
✅ Inserted batch 50
✅ Inserte

In [57]:
int_cat=[]
float_cat=[]
object_cat=[]
for col in df.columns:
    if df[col].dtype=='int64':
        int_cat.append(col)
    elif df[col].dtype=='float64':
        float_cat.append(col)
    elif df[col].dtype=='object':
        object_cat.append(col)
        
boolean_cat = []
categorical_cat = []
for col in object_cat:
    # Check if all non-null values are boolean
    if df[col].isin([True, False]).all():
        boolean_cat.append(col)
    else:
        categorical_cat.append(col)
            

In [58]:
int_cat

['ID',
 'Seniority',
 'Policies_in_force',
 'Max_policies',
 'Max_products',
 'Lapse',
 'Payment',
 'N_claims_year',
 'N_claims_history',
 'Type_risk',
 'Area',
 'Second_driver',
 'Year_matriculation',
 'Power',
 'Cylinder_capacity',
 'N_doors',
 'Weight']

In [59]:
float_cat

['Premium', 'Cost_claims_year', 'R_Claims_history', 'Value_vehicle', 'Length']

In [60]:
boolean_cat

[]

In [62]:
categorical_cat  #obj type columns  , including date columns

['Date_start_contract',
 'Date_last_renewal',
 'Date_next_renewal',
 'Date_birth',
 'Date_driving_licence',
 'Distribution_channel',
 'Date_lapse',
 'Type_fuel']

In [63]:
df.columns

Index(['ID', 'Date_start_contract', 'Date_last_renewal', 'Date_next_renewal',
       'Date_birth', 'Date_driving_licence', 'Distribution_channel',
       'Seniority', 'Policies_in_force', 'Max_policies', 'Max_products',
       'Lapse', 'Date_lapse', 'Payment', 'Premium', 'Cost_claims_year',
       'N_claims_year', 'N_claims_history', 'R_Claims_history', 'Type_risk',
       'Area', 'Second_driver', 'Year_matriculation', 'Power',
       'Cylinder_capacity', 'Value_vehicle', 'N_doors', 'Type_fuel', 'Length',
       'Weight'],
      dtype='object')

In [64]:
count=0
for entry in df['Cost_claims_year']:
    if entry==0:
       count+=1
print(f"Number of zero entries in Cost_claims_year: {count}") 
print(f"Percentage of zero entries in Cost_claims_year: {count/len(df)*100:.2f}%")  
df['Cost_claims_year'].describe()

Number of zero entries in Cost_claims_year: 85909
Percentage of zero entries in Cost_claims_year: 81.39%


count    105555.000000
mean        153.557305
std        1477.112362
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max      260853.240000
Name: Cost_claims_year, dtype: float64

In [65]:
target1=df['Cost_claims_year']
df.drop(columns=['Cost_claims_year','ID','R_Claims_history'],inplace=True)


In [66]:
from scipy.stats import skew
from sklearn.preprocessing import PowerTransformer


In [68]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df, target1, test_size=0.2, random_state=42)

# Only apply to numeric columns
numeric_cols = X_train.select_dtypes(include=['int64', 'float64']).columns

for col in numeric_cols:
    if skew(X_train[col]) > 0.5:
        pt = PowerTransformer(method='yeo-johnson')
        X_train[col] = pt.fit_transform(X_train[[col]])
        X_test[col] = pt.transform(X_test[[col]])
        print(f"Applied PowerTransformer to {col} due to skewness: {skew(X_train[col])}")
    else:
        print(f"No transformation needed for {col}, skewness: {skew(X_train[col])}")

Applied PowerTransformer to Seniority due to skewness: 0.05444350838316816
Applied PowerTransformer to Policies_in_force due to skewness: 0.9456679426666397
Applied PowerTransformer to Max_policies due to skewness: 0.2673094396251911
Applied PowerTransformer to Max_products due to skewness: 3.6746648507883894
Applied PowerTransformer to Lapse due to skewness: 1.4727176722279207
Applied PowerTransformer to Payment due to skewness: 0.7743166115103545
Applied PowerTransformer to Premium due to skewness: 0.09273798344438587
Applied PowerTransformer to N_claims_year due to skewness: 1.6088909660068909
Applied PowerTransformer to N_claims_history due to skewness: 0.11403836469619968
No transformation needed for Type_risk, skewness: -1.8358140021506546
Applied PowerTransformer to Area due to skewness: 1.0143441555459956
Applied PowerTransformer to Second_driver due to skewness: 2.275255479076479
No transformation needed for Year_matriculation, skewness: -1.4255839830468506
No transformation n

In [69]:
def remove_outliers(df, threshold=3):
    """
    Remove outliers from the DataFrame based on Z-score.
    """
    from scipy import stats
    z_scores = np.abs(stats.zscore(df.select_dtypes(include=[np.number])))
    return df[(z_scores < threshold).all(axis=1)]
X_train = remove_outliers(X_train)

In [80]:
import pandas as pd
import numpy as np

def feature_engineering(df):
    """
    Perform feature engineering on the DataFrame.
    """
    df = df.copy()
    today = pd.to_datetime("today")

    # Convert relevant date columns safely
    df['Date_lapse'] = pd.to_datetime(df['Date_lapse'], dayfirst=True, errors='coerce')
    df['Date_start_contract'] = pd.to_datetime(df['Date_start_contract'], dayfirst=True, errors='coerce')
    df['Date_driving_licence'] = pd.to_datetime(df['Date_driving_licence'], dayfirst=True, errors='coerce')
    df['Date_last_renewal'] = pd.to_datetime(df['Date_last_renewal'], dayfirst=True, errors='coerce')
    df['Date_next_renewal'] = pd.to_datetime(df['Date_next_renewal'], dayfirst=True, errors='coerce')

    # Age since lapse (if lapsed)
    df['Age'] = ((today - df['Date_lapse']).dt.days // 365).fillna(0).astype(int)

    # Vehicle age
    df['Vehicle_age'] = today.year - df['Year_matriculation'] if 'Year_matriculation' in df.columns else np.nan

    # Insurance duration (from start to lapse or today if not lapsed)
    df['Insurance_duration'] = ((df['Date_lapse'].fillna(today) - df['Date_start_contract']).dt.days//30).fillna(0).astype(int)

    # Licence age
    df['Licence_age'] = ((today - df['Date_driving_licence']).dt.days // 365).fillna(0).astype(int)

    # Insurance status (1 = active, 0 = terminated)
    df['Insurance_status'] = df['Date_lapse'].isna().astype(int)

    # Time between renewals
    df['next_claim_duration'] = (
        (df['Date_next_renewal'] - df['Date_last_renewal']).dt.days
    ).fillna(0).astype(int)

    # Drop unneeded date columns
    df = df.drop(columns=[
        'Date_start_contract',
        'Date_last_renewal',
        'Date_next_renewal',
        'Date_driving_licence',
        'Date_birth',
        'Date_lapse'
    ], errors='ignore')

    return df
x_train_engineered = feature_engineering(X_train)

In [81]:
x_train_engineered.head()

,Distribution_channel,Seniority,Policies_in_force,Max_policies,Max_products,Lapse,Payment,Premium,N_claims_year,N_claims_history,...,N_doors,Type_fuel,Length,Weight,Age,Vehicle_age,Insurance_duration,Licence_age,Insurance_status,next_claim_duration
95053,1,0.117199,-0.636621,1.271643,-2.959438e-15,-0.505509,1.459488,0.234469,2.043115,0.641536,...,4,D,1.121606,1.068384,0,22,132,33,0,365
98446,1,-0.523140,-0.636621,-0.968268,-2.959438e-15,-0.505509,1.459488,1.091568,2.122085,0.325574,...,3,P,0.176213,-0.090149,0,16,106,22,0,365
41836,1,-1.821256,-0.636621,-0.968268,-2.959438e-15,-0.505509,-0.685172,-0.640997,-0.479122,-1.179999,...,3,D,-1.597379,-0.664788,8,24,11,35,0,365
16050,1,-0.523140,1.470201,1.271643,-2.959438e-15,-0.505509,1.459488,1.119254,-0.479122,0.641536,...,5,D,0.195369,0.197195,0,21,97,47,0,365
39930,0,-0.523140,-0.636621,-0.968268,-2.959438e-15,-0.505509,-0.685172,-1.872364,-0.479122,-1.179999,...,0,P,0.076824,-2.224009,0,10,114,28,0,366


AttributeError: 'int' object has no attribute 'dtype'